# Tugas 3: Pipeline Pemrosesan ECG
## Full ECG Processing Pipeline

**Mata Kuliah:** Pengolahan Sinyal Medis -- Universitas Indonesia

**Tenggat:** Akhir Minggu 6

---

| | |
|---|---|
| **Nama Mahasiswa** | *(isi nama Anda di sini)* |
| **NPM** | *(isi NPM Anda di sini)* |

---

### Petunjuk Pengerjaan

1. Lengkapi semua bagian yang ditandai dengan `# TODO`.
2. Jawab semua pertanyaan analisis pada sel Markdown yang telah disediakan.
3. Pastikan semua sel kode dapat dijalankan dari awal sampai akhir tanpa error (`Kernel > Restart & Run All`).
4. Kumpulkan notebook ini (file `.ipynb`) sesuai instruksi yang diberikan.

> **Catatan Khusus:** Tugas ini merupakan transisi menuju pengembangan pipeline sebagai script Python. Setelah menyelesaikan notebook ini, Anda akan mengimplementasikan ulang pipeline yang sama sebagai script Python standalone.

### Kriteria Penilaian

| Komponen | Bobot |
|---|---|
| Bagian 1: Memuat dan Visualisasi Data ECG | 10% |
| Bagian 2: Preprocessing ECG | 20% |
| Bagian 3: Deteksi R-Peak (Pan-Tompkins) | 25% |
| Bagian 4: Analisis RR Interval dan HRV | 25% |
| Pertanyaan Analisis | 20% |

In [ ]:
# Impor library yang dibutuhkan
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig

from medsinyal.io import load_synthetic
from medsinyal.viz import plot_signal, plot_ecg
from medsinyal.filters import bandpass_filter, notch_filter, remove_baseline_wander
from medsinyal.ecg import (
    preprocess_ecg,
    detect_r_peaks,
    compute_rr_intervals,
    compute_heart_rate,
    compute_hrv_features,
    extract_ecg_features,
)

# Konfigurasi plot
plt.rcParams['figure.figsize'] = (14, 4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

---
## Bagian 1: Memuat dan Visualisasi Data ECG (10%)
### Loading and Visualizing ECG Data

Pada bagian ini, Anda akan memuat data ECG sintetik dan memahami struktur datanya sebelum membangun pipeline pemrosesan secara lengkap.

**Gambaran Pipeline ECG:**
Pipeline yang akan Anda bangun mencakup empat tahap utama:

1. **Pemuatan & Visualisasi Data** -- memahami sinyal mentah (raw)
2. **Preprocessing** -- menghilangkan noise dan artefak
3. **Deteksi R-Peak** -- menemukan kompleks QRS menggunakan algoritma Pan-Tompkins
4. **Analisis HRV** -- menghitung fitur Heart Rate Variability dari interval RR

Data yang tersedia adalah rekaman ECG sintetik yang mengandung sinyal bersih (*clean*) dan sinyal bernoise (*noisy*) beserta anotasi R-peak sebagai *ground truth*.

**Tugas 1.1:** Muat data ECG sintetik menggunakan `load_synthetic('synthetic_ecg')`. Cetak semua *keys* yang tersedia beserta *shape*-nya. Tampilkan informasi dasar: sampling rate, durasi rekaman, jumlah sampel, dan *expected heart rate* dari metadata.

In [ ]:
# TODO: Muat data ECG sintetik
# ecg_data = load_synthetic('synthetic_ecg')

# TODO: Cetak semua keys dan shape-nya
# print("Keys dalam file:")
# for key in ecg_data:
#     if isinstance(ecg_data[key], np.ndarray):
#         print(f"  {key}: shape={ecg_data[key].shape}, dtype={ecg_data[key].dtype}")
#     else:
#         print(f"  {key}: {ecg_data[key]}")

# TODO: Ekstrak variabel utama dari ecg_data
# t = ecg_data['t']
# fs = float(ecg_data['fs'])
# ecg_clean = ecg_data['ecg_clean']
# ecg_noisy = ecg_data['ecg_noisy']
# r_peak_indices = ecg_data['r_peak_indices']   # Ground truth R-peak indices
# heart_rate_meta = ecg_data['heart_rate']      # Expected heart rate dari metadata

# TODO: Cetak informasi dasar rekaman
# duration = len(t) / fs
# print(f"\nInformasi Rekaman:")
# print(f"  Sampling rate     : {fs} Hz")
# print(f"  Jumlah sampel     : {len(t)}")
# print(f"  Durasi rekaman    : {duration:.2f} detik")
# print(f"  Expected HR (meta): {heart_rate_meta} BPM")
# print(f"  Jumlah R-peak (GT): {len(r_peak_indices)}")


**Tugas 1.2:** Plot 5 detik pertama dari `ecg_clean` dan `ecg_noisy` secara berdampingan (2 subplot). Identifikasi secara visual jenis-jenis noise yang terlihat:
- **Baseline wander**: pergeseran lambat pada garis dasar sinyal (frekuensi < 0.5 Hz)
- **Noise frekuensi tinggi (50 Hz)**: osilasi cepat yang terlihat sebagai "bulu" halus pada sinyal (*power line interference*)
- **Random noise**: variasi acak yang tersebar merata

In [ ]:
# TODO: Tentukan batas indeks untuk 5 detik pertama
# n_5sec = int(5 * fs)   # Jumlah sampel dalam 5 detik
# t_5sec = t[:n_5sec]
# ecg_clean_5sec = ecg_clean[:n_5sec]
# ecg_noisy_5sec = ecg_noisy[:n_5sec]

# TODO: Plot ECG bersih dan bernoise dalam 2 subplot
# fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)
#
# axes[0].plot(t_5sec, ecg_clean_5sec, 'b-', linewidth=0.8)
# axes[0].set_title('ECG Bersih (Clean) -- 5 Detik Pertama')
# axes[0].set_ylabel('Amplitudo (mV)')
#
# axes[1].plot(t_5sec, ecg_noisy_5sec, 'r-', linewidth=0.8)
# axes[1].set_title('ECG Bernoise (Noisy) -- 5 Detik Pertama')
# axes[1].set_ylabel('Amplitudo (mV)')
# axes[1].set_xlabel('Waktu (detik)')
#
# plt.tight_layout()
# plt.show()

# TODO: Cetak observasi Anda tentang jenis noise yang terlihat
# print("Observasi noise yang terlihat pada sinyal:")
# print("  - Baseline wander  : ...")
# print("  - Power line (50Hz): ...")
# print("  - Random noise     : ...")


---
## Bagian 2: Preprocessing ECG (20%)
### ECG Preprocessing

Preprocessing ECG bertujuan menghilangkan artefak dan noise agar sinyal siap untuk analisis lebih lanjut. Tiga tahap utama preprocessing:

1. **Notch filter 50 Hz** -- menghilangkan *power line interference* (gangguan jala-jala listrik PLN)
2. **Highpass / Baseline removal 0.5 Hz** -- menghilangkan *baseline wander* akibat gerakan atau pernapasan
3. **Bandpass filter 0.5--40 Hz** -- membatasi bandwidth sinyal ke rentang fisiologis ECG

Urutan penerapan filter ini penting dan akan menjadi topik salah satu pertanyaan analisis.

**Tugas 2.1:** Implementasikan preprocessing langkah demi langkah secara manual (tanpa menggunakan wrapper `preprocess_ecg`). Terapkan ketiga filter secara berurutan dan plot hasilnya dalam 4 subplot (sinyal asli + 3 tahap).

In [ ]:
# TODO: Step 1 -- Terapkan notch filter untuk menghilangkan power line 50 Hz
# ecg_notched = notch_filter(ecg_noisy, 50, fs)

# TODO: Step 2 -- Hilangkan baseline wander
# ecg_no_baseline = remove_baseline_wander(ecg_notched, fs)

# TODO: Step 3 -- Terapkan bandpass filter 0.5-40 Hz
# ecg_preprocessed = bandpass_filter(ecg_no_baseline, 0.5, 40, fs)

# TODO: Plot keempat sinyal (asli + 3 tahap) dalam 4 subplot
# fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)
#
# n_plot = int(5 * fs)  # Plot 5 detik pertama
#
# axes[0].plot(t[:n_plot], ecg_noisy[:n_plot], 'r-', linewidth=0.7)
# axes[0].set_title('Sinyal Asli (Noisy)')
# axes[0].set_ylabel('Amplitudo (mV)')
#
# axes[1].plot(t[:n_plot], ecg_notched[:n_plot], color='orange', linewidth=0.7)
# axes[1].set_title('Setelah Notch Filter 50 Hz')
# axes[1].set_ylabel('Amplitudo (mV)')
#
# axes[2].plot(t[:n_plot], ecg_no_baseline[:n_plot], color='purple', linewidth=0.7)
# axes[2].set_title('Setelah Baseline Removal (Highpass 0.5 Hz)')
# axes[2].set_ylabel('Amplitudo (mV)')
#
# axes[3].plot(t[:n_plot], ecg_preprocessed[:n_plot], 'g-', linewidth=0.7)
# axes[3].set_title('Setelah Bandpass Filter 0.5--40 Hz (Hasil Akhir Preprocessing)')
# axes[3].set_ylabel('Amplitudo (mV)')
# axes[3].set_xlabel('Waktu (detik)')
#
# plt.tight_layout()
# plt.show()


**Tugas 2.2:** Evaluasi kualitas preprocessing secara kuantitatif. Hitung RMSE dan SNR di setiap tahap dibandingkan terhadap `ecg_clean` sebagai referensi. Tampilkan hasilnya dalam bentuk tabel.

**Rumus SNR:**

$$\text{SNR}_{\text{dB}} = 10 \cdot \log_{10}\left(\frac{P_{\text{signal}}}{P_{\text{noise}}}\right)$$

di mana $P_{\text{noise}} = \text{mean}((\text{signal} - \text{ecg\_clean})^2)$ dan $P_{\text{signal}} = \text{mean}(\text{ecg\_clean}^2)$.

In [ ]:
# TODO: Hitung RMSE dan SNR di setiap tahap preprocessing
# def compute_metrics(signal, reference):
#     noise = signal - reference
#     rmse = np.sqrt(np.mean(noise**2))
#     snr = 10 * np.log10(np.mean(reference**2) / np.mean(noise**2))
#     return rmse, snr
#
# signals = {
#     'Noisy asli'              : ecg_noisy,
#     'Setelah notch filter'    : ecg_notched,
#     'Setelah baseline removal': ecg_no_baseline,
#     'Setelah bandpass filter' : ecg_preprocessed,
# }
#
# print(f"{'Tahap':<32} {'RMSE':>12} {'SNR (dB)':>12}")
# print("-" * 58)
# for name, s in signals.items():
#     rmse, snr = compute_metrics(s, ecg_clean)
#     print(f"{name:<32} {rmse:>12.6f} {snr:>12.2f}")


**Tugas 2.3:** Bandingkan hasil preprocessing manual Anda dengan wrapper `preprocess_ecg(ecg_noisy, fs)` dari `medsinyal`. Hitung selisih antara kedua sinyal. Hasilnya seharusnya mendekati nol jika implementasi Anda benar.

In [ ]:
# TODO: Gunakan wrapper preprocess_ecg dari medsinyal
# ecg_preprocessed_wrapper = preprocess_ecg(ecg_noisy, fs)

# TODO: Hitung selisih antara hasil manual dan wrapper
# diff = ecg_preprocessed - ecg_preprocessed_wrapper
# max_diff = np.max(np.abs(diff))
# rms_diff = np.sqrt(np.mean(diff**2))

# TODO: Cetak hasil perbandingan
# print("Perbandingan Preprocessing Manual vs Wrapper:")
# print(f"  Max absolute difference : {max_diff:.8f}")
# print(f"  RMS difference          : {rms_diff:.8f}")
# if max_diff < 1e-6:
#     print("  Kesimpulan: Hasil IDENTIK -- implementasi manual benar.")
# else:
#     print("  Kesimpulan: Ada perbedaan kecil -- cek urutan/parameter filter.")

# TODO: Plot perbandingan untuk 3 detik pertama
# n_3sec = int(3 * fs)
# fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)
# axes[0].plot(t[:n_3sec], ecg_preprocessed[:n_3sec], 'g-', label='Manual')
# axes[0].plot(t[:n_3sec], ecg_preprocessed_wrapper[:n_3sec], 'b--', label='Wrapper', alpha=0.7)
# axes[0].set_title('Perbandingan Hasil Preprocessing Manual vs Wrapper')
# axes[0].set_ylabel('Amplitudo (mV)')
# axes[0].legend()
# axes[1].plot(t[:n_3sec], diff[:n_3sec], 'r-')
# axes[1].set_title('Selisih (Manual - Wrapper)')
# axes[1].set_ylabel('Selisih')
# axes[1].set_xlabel('Waktu (detik)')
# plt.tight_layout()
# plt.show()


---
## Bagian 3: Deteksi R-Peak (Pan-Tompkins) (25%)
### R-Peak Detection

Algoritma **Pan-Tompkins** (1985) adalah metode klasik dan masih banyak digunakan untuk deteksi kompleks QRS pada sinyal ECG. Algoritma ini bekerja dalam lima langkah:

1. **Bandpass filter 5--15 Hz** -- memperkuat energi QRS dan melemahkan gelombang P, T, dan noise
2. **Diferensiasi** -- memperkuat kemiringan (*slope*) tajam dari kompleks QRS
3. **Kuadrasi (*squaring*)** -- penguatan non-linear untuk memperbesar perbedaan QRS vs non-QRS
4. **Moving window integration** -- memperhalus sinyal dan menghasilkan envelope QRS
5. **Deteksi puncak adaptif** -- menemukan puncak dengan threshold yang menyesuaikan diri

Deteksi R-peak yang akurat sangat penting karena semua analisis HRV didasarkan pada posisi R-peak.

**Tugas 3.1:** Implementasikan versi sederhana algoritma Pan-Tompkins secara manual. Plot sinyal antara (*intermediate signals*) dari setiap langkah untuk 3 detik pertama dalam 4 subplot.

In [ ]:
# TODO: Implementasikan Pan-Tompkins secara manual

# Step 1: Bandpass filter 5-15 Hz untuk memperkuat energi QRS
# ecg_bp = bandpass_filter(ecg_preprocessed, 5.0, 15.0, fs, order=2)

# Step 2: Diferensiasi -- menghitung turunan untuk memperkuat kemiringan QRS
# ecg_diff = np.diff(ecg_bp)
# Tambahkan satu elemen agar panjang sama dengan sinyal asli
# ecg_diff = np.append(ecg_diff, ecg_diff[-1])

# Step 3: Kuadrasi -- penguatan non-linear
# ecg_squared = ecg_diff ** 2

# Step 4: Moving window integration (window = 150 ms)
# window_samples = int(0.15 * fs)  # 150 ms dalam sampel
# kernel = np.ones(window_samples) / window_samples
# ecg_integrated = np.convolve(ecg_squared, kernel, mode='same')

# TODO: Plot sinyal intermediate untuk 3 detik pertama
# n_3sec = int(3 * fs)
# fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)
#
# axes[0].plot(t[:n_3sec], ecg_bp[:n_3sec], 'b-', linewidth=0.8)
# axes[0].set_title('Step 1: Bandpass Filter 5-15 Hz')
# axes[0].set_ylabel('Amplitudo')
#
# axes[1].plot(t[:n_3sec], ecg_diff[:n_3sec], 'orange', linewidth=0.8)
# axes[1].set_title('Step 2: Diferensiasi')
# axes[1].set_ylabel('Amplitudo')
#
# axes[2].plot(t[:n_3sec], ecg_squared[:n_3sec], 'r-', linewidth=0.8)
# axes[2].set_title('Step 3: Kuadrasi')
# axes[2].set_ylabel('Amplitudo')
#
# axes[3].plot(t[:n_3sec], ecg_integrated[:n_3sec], 'g-', linewidth=0.8)
# axes[3].set_title('Step 4: Moving Window Integration (150 ms)')
# axes[3].set_ylabel('Amplitudo')
# axes[3].set_xlabel('Waktu (detik)')
#
# plt.tight_layout()
# plt.show()

# TODO (Opsional): Step 5 -- Deteksi puncak sederhana pada sinyal terintegrasi
# threshold = 0.6 * np.max(ecg_integrated)
# min_distance = int(0.2 * fs)  # Minimum jarak antar R-peak: 200 ms
# r_peaks_manual, _ = sig.find_peaks(ecg_integrated, height=threshold, distance=min_distance)
# print(f"Jumlah R-peak terdeteksi (manual): {len(r_peaks_manual)}")


**Tugas 3.2:** Gunakan fungsi `detect_r_peaks(ecg_preprocessed, fs)` dari `medsinyal` untuk mendeteksi R-peak. Plot hasilnya menggunakan `plot_ecg` dengan marker R-peak.

In [ ]:
# TODO: Deteksi R-peak menggunakan fungsi dari medsinyal
# r_peaks_detected = detect_r_peaks(ecg_preprocessed, fs)
# print(f"Jumlah R-peak terdeteksi: {len(r_peaks_detected)}")
# print(f"Jumlah R-peak ground truth: {len(r_peak_indices)}")

# TODO: Plot ECG preprocessed dengan R-peak menggunakan plot_ecg
# plot_ecg(
#     t,
#     ecg_preprocessed,
#     r_peaks=r_peaks_detected,
#     title='ECG Preprocessed dengan R-peak Terdeteksi'
# )

# TODO: Tampilkan juga posisi waktu beberapa R-peak pertama
# print("\nPosisi 5 R-peak pertama yang terdeteksi:")
# for i, idx in enumerate(r_peaks_detected[:5]):
#     print(f"  R-peak {i+1}: indeks={idx}, waktu={t[idx]:.3f} detik")


**Tugas 3.3:** Evaluasi performa deteksi R-peak dengan membandingkan hasil deteksi terhadap *ground truth* `r_peak_indices`. Gunakan toleransi 50 ms untuk menentukan *True Positive*.

**Metrik yang dihitung:**
- **TP** (*True Positive*): R-peak terdeteksi yang memiliki pasangan di ground truth (dalam toleransi 50 ms)
- **FP** (*False Positive*): R-peak terdeteksi yang tidak memiliki pasangan di ground truth
- **FN** (*False Negative*): R-peak ground truth yang tidak terdeteksi
- **Sensitivity** = TP / (TP + FN) × 100%
- **PPV** (*Positive Predictive Value*) = TP / (TP + FP) × 100%

In [ ]:
# TODO: Evaluasi performa deteksi R-peak
# tolerance_samples = int(0.05 * fs)  # Toleransi 50 ms dalam sampel
#
# tp = 0
# matched_gt = set()  # Set untuk melacak ground truth yang sudah matched
#
# for det_peak in r_peaks_detected:
#     # Cek apakah ada ground truth peak dalam radius toleransi
#     for j, gt_peak in enumerate(r_peak_indices):
#         if j not in matched_gt and abs(det_peak - gt_peak) <= tolerance_samples:
#             tp += 1
#             matched_gt.add(j)
#             break
#
# fp = len(r_peaks_detected) - tp
# fn = len(r_peak_indices) - tp
#
# sensitivity = tp / (tp + fn) * 100 if (tp + fn) > 0 else 0
# ppv = tp / (tp + fp) * 100 if (tp + fp) > 0 else 0
#
# print("Evaluasi Performa Deteksi R-Peak (toleransi = 50 ms):")
# print(f"  True Positives (TP)  : {tp}")
# print(f"  False Positives (FP) : {fp}")
# print(f"  False Negatives (FN) : {fn}")
# print(f"  Sensitivity          : {sensitivity:.2f}%")
# print(f"  PPV (Precision)      : {ppv:.2f}%")


---
## Bagian 4: Analisis RR Interval dan HRV (25%)
### RR Interval and HRV Analysis

**RR interval** adalah jarak waktu antara dua R-peak yang berurutan, dan merupakan dasar dari analisis *Heart Rate Variability* (HRV). Nilai RR interval normal saat istirahat berkisar antara **0.6--1.0 detik** (60--100 BPM).

HRV mencerminkan aktivitas sistem saraf otonom (ANS) yang mengatur jantung. Beberapa fitur HRV yang penting:

| Fitur | Deskripsi | Nilai Normal (istirahat) |
|---|---|---|
| SDNN | Std dev semua interval RR (ms) | 50--100 ms |
| RMSSD | Root mean square successive differences (ms) | 20--50 ms |
| pNN50 | % interval RR berurutan yang berbeda > 50 ms | 10--40% |
| mean_hr | Heart rate rata-rata (BPM) | 60--100 BPM |

**Tugas 4.1:** Hitung RR interval menggunakan `compute_rr_intervals(r_peaks_detected, fs)` dan *instantaneous heart rate* menggunakan `compute_heart_rate(rr_intervals)`. Plot takogram RR interval dan *instantaneous heart rate* dengan garis referensi nilai normal.

In [ ]:
# TODO: Hitung RR intervals
# rr_intervals = compute_rr_intervals(r_peaks_detected, fs)
# print(f"Jumlah RR interval: {len(rr_intervals)}")
# print(f"RR interval rata-rata: {np.mean(rr_intervals)*1000:.1f} ms")
# print(f"RR interval min: {np.min(rr_intervals)*1000:.1f} ms")
# print(f"RR interval max: {np.max(rr_intervals)*1000:.1f} ms")

# TODO: Hitung instantaneous heart rate
# hr_instantaneous = compute_heart_rate(rr_intervals)

# TODO: Plot RR interval tachogram dan instantaneous heart rate dalam 2 subplot
# beat_numbers = np.arange(1, len(rr_intervals) + 1)
#
# fig, axes = plt.subplots(2, 1, figsize=(14, 8))
#
# axes[0].plot(beat_numbers, rr_intervals * 1000, 'b-o', markersize=3, linewidth=0.8)
# axes[0].axhline(600, color='g', linestyle='--', alpha=0.7, label='Batas bawah normal (600 ms)')
# axes[0].axhline(1000, color='g', linestyle='--', alpha=0.7, label='Batas atas normal (1000 ms)')
# axes[0].set_title('Takogram RR Interval')
# axes[0].set_xlabel('Nomor Beat')
# axes[0].set_ylabel('RR Interval (ms)')
# axes[0].legend()
#
# axes[1].plot(beat_numbers, hr_instantaneous, 'r-o', markersize=3, linewidth=0.8)
# axes[1].axhline(60, color='g', linestyle='--', alpha=0.7, label='Batas bawah normal (60 BPM)')
# axes[1].axhline(100, color='g', linestyle='--', alpha=0.7, label='Batas atas normal (100 BPM)')
# axes[1].set_title('Instantaneous Heart Rate')
# axes[1].set_xlabel('Nomor Beat')
# axes[1].set_ylabel('Heart Rate (BPM)')
# axes[1].legend()
#
# plt.tight_layout()
# plt.show()


**Tugas 4.2:** Hitung fitur-fitur HRV menggunakan `compute_hrv_features(rr_intervals)`. Tampilkan semua fitur dalam tabel berformat rapi dan interpretasikan nilainya terhadap rentang normal.

In [ ]:
# TODO: Hitung fitur HRV
# hrv_features = compute_hrv_features(rr_intervals)

# TODO: Tampilkan semua fitur dalam tabel berformat
# print("Fitur HRV yang Dihitung:")
# print("-" * 55)
# print(f"{'Fitur':<20} {'Nilai':>10} {'Satuan':>8} {'Normal':>15}")
# print("-" * 55)
# print(f"{'mean_rr':<20} {hrv_features.get('mean_rr', 0)*1000:>10.1f} {'ms':>8} {'600-1000 ms':>15}")
# print(f"{'SDNN':<20} {hrv_features.get('sdnn', 0)*1000:>10.1f} {'ms':>8} {'50-100 ms':>15}")
# print(f"{'RMSSD':<20} {hrv_features.get('rmssd', 0)*1000:>10.1f} {'ms':>8} {'20-50 ms':>15}")
# print(f"{'pNN50':<20} {hrv_features.get('pnn50', 0):>10.1f} {'%':>8} {'10-40 %':>15}")
# print(f"{'mean_hr':<20} {hrv_features.get('mean_hr', 0):>10.1f} {'BPM':>8} {'60-100 BPM':>15}")
# print(f"{'std_hr':<20} {hrv_features.get('std_hr', 0):>10.1f} {'BPM':>8} {'-':>15}")
# print("-" * 55)

# TODO: Interpretasikan nilai SDNN dan RMSSD
# Apakah nilai SDNN Anda dalam rentang normal (50-100 ms)?
# Apakah nilai RMSSD Anda dalam rentang normal (20-50 ms)?
# sdnn_val = hrv_features.get('sdnn', 0) * 1000
# rmssd_val = hrv_features.get('rmssd', 0) * 1000
# print(f"\nInterpretasi:")
# print(f"  SDNN = {sdnn_val:.1f} ms --> {'Normal' if 50 <= sdnn_val <= 100 else 'Di luar rentang normal'}")
# print(f"  RMSSD = {rmssd_val:.1f} ms --> {'Normal' if 20 <= rmssd_val <= 50 else 'Di luar rentang normal'}")


**Tugas 4.3 (Bonus):** Hitung seluruh fitur ECG menggunakan `extract_ecg_features(ecg_preprocessed, r_peaks_detected, fs)`. Cetak dictionary fitur yang dihasilkan dan jelaskan fitur mana saja yang berpotensi digunakan untuk klasifikasi aritmia.

In [ ]:
# TODO (Bonus): Hitung seluruh fitur ECG menggunakan extract_ecg_features
# ecg_features = extract_ecg_features(ecg_preprocessed, r_peaks_detected, fs)

# TODO: Cetak semua fitur yang dihasilkan
# print("Fitur ECG Lengkap (dari extract_ecg_features):")
# print("-" * 50)
# for feat_name, feat_val in ecg_features.items():
#     print(f"  {feat_name:<30} : {feat_val}")

# TODO: Jelaskan fitur-fitur yang relevan untuk klasifikasi aritmia
# Fitur HRV (SDNN, RMSSD, pNN50) mencerminkan variabilitas denyut jantung
# Fitur morfologi QRS (durasi, amplitudo) menunjukkan bentuk kompleks QRS
# Tulis analisis Anda di bawah ini:
# print("\nFitur yang relevan untuk klasifikasi aritmia:")
# print("  - ...")


---
## Pertanyaan Analisis (20%)
### Analysis Questions

Jawab pertanyaan-pertanyaan berikut berdasarkan pemahaman Anda dari hasil percobaan dan materi kuliah. Jawab dengan lengkap dan jelas.

### Pertanyaan 1

Dalam preprocessing ECG, urutan penerapan filter sangat penting. Mengapa lebih baik menerapkan *notch filter* (50 Hz) **sebelum** *highpass filter* (*baseline removal*), bukan sebaliknya? Jelaskan dengan mempertimbangkan karakteristik frekuensi masing-masing filter dan bagaimana urutan yang berbeda dapat mempengaruhi hasil preprocessing.

**Jawaban:**

*(Tulis jawaban Anda di sini)*


### Pertanyaan 2

Algoritma Pan-Tompkins menggunakan kombinasi *bandpass filter*, *differentiator*, dan *squaring* sebelum integrasi. Jelaskan peran masing-masing langkah:

**(a)** Mengapa *bandpass filter* 5--15 Hz dipilih untuk memperkuat kompleks QRS? Apa yang terjadi pada gelombang P dan T setelah filter ini?

**(b)** Apa peran *differentiator* dalam mendeteksi puncak R? Mengapa kemiringan (*slope*) sinyal penting?

**(c)** Mengapa *squaring* dilakukan sebelum integrasi? Apa efek non-linear yang dihasilkan?

**Jawaban:**

*(a)* *(Tulis jawaban Anda di sini)*

*(b)* *(Tulis jawaban Anda di sini)*

*(c)* *(Tulis jawaban Anda di sini)*


### Pertanyaan 3

Berdasarkan hasil analisis HRV Anda di Bagian 4:

**(a)** Berapa heart rate rata-rata yang Anda dapatkan? Apakah termasuk dalam rentang normal (60--100 BPM)?

**(b)** Bagaimana Anda menginterpretasikan nilai SDNN dan RMSSD yang Anda hitung? Apakah keduanya dalam rentang normal?

**(c)** Dalam konteks klinis, kondisi apa yang dapat menyebabkan RMSSD **sangat rendah** (< 15 ms)? Dan kondisi apa yang menyebabkan RMSSD **sangat tinggi** (> 100 ms)? Berikan contoh masing-masing.

**Jawaban:**

*(a)* *(Tulis jawaban Anda di sini)*

*(b)* *(Tulis jawaban Anda di sini)*

*(c)* *(Tulis jawaban Anda di sini)*


---
## Bonus: Transisi ke Script Python
### Transition to Python Script

Setelah menyelesaikan notebook ini, implementasikan ulang pipeline ECG Anda sebagai script Python standalone. Script ini merupakan langkah awal transisi dari *exploratory notebook* menuju kode produksi yang terstruktur.

**Spesifikasi script `ecg_pipeline.py`:**
- **Input**: path ke file `.npz` ECG (via argumen `--input`)
- **Output**: print ringkasan ke terminal (heart rate, fitur HRV, jumlah R-peak terdeteksi)
- **Simpan** di: `assignments/tugas03_ecg_pipeline/ecg_pipeline.py`

**Contoh penggunaan:**
```bash
python ecg_pipeline.py --input data/synthetic/synthetic_ecg.npz
```

**Contoh output yang diharapkan:**
```
=== ECG Pipeline Summary ===
File         : synthetic_ecg.npz
Duration     : 30.00 s
R-peaks      : 42
Mean HR      : 72.5 BPM
SDNN         : 65.3 ms
RMSSD        : 38.1 ms
pNN50        : 18.4 %
===========================
```

Ini adalah latihan untuk transisi dari notebook ke script produksi yang akan lebih sering digunakan di paruh kedua kuliah, terutama saat membangun pipeline pemrosesan sinyal yang dapat dijalankan secara otomatis (*batch processing*).

In [ ]:
# TODO (Bonus): Buat kerangka script ecg_pipeline.py
# Salin kode di bawah ini ke file ecg_pipeline.py dan lengkapi bagian yang kosong

script_template = '''
#!/usr/bin/env python
"""ECG Processing Pipeline Script.

Usage:
    python ecg_pipeline.py --input data/synthetic/synthetic_ecg.npz
"""
import argparse
import numpy as np
from medsinyal.io import load_synthetic
from medsinyal.ecg import preprocess_ecg, detect_r_peaks, compute_rr_intervals
from medsinyal.ecg import compute_heart_rate, compute_hrv_features


def run_pipeline(input_path: str) -> None:
    """Jalankan pipeline ECG dan cetak ringkasan."""
    # TODO: Muat data
    # data = ...

    # TODO: Ekstrak variabel
    # fs = ...
    # ecg_noisy = ...
    # t = ...

    # TODO: Preprocessing
    # ecg_preprocessed = preprocess_ecg(ecg_noisy, fs)

    # TODO: Deteksi R-peak
    # r_peaks = detect_r_peaks(ecg_preprocessed, fs)

    # TODO: Hitung RR interval dan HRV
    # rr = compute_rr_intervals(r_peaks, fs)
    # hrv = compute_hrv_features(rr)

    # TODO: Cetak ringkasan
    # print("=== ECG Pipeline Summary ===")
    # ...
    pass


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="ECG Processing Pipeline")
    parser.add_argument("--input", required=True, help="Path ke file .npz ECG")
    args = parser.parse_args()
    run_pipeline(args.input)
'''

print("Kerangka script ecg_pipeline.py:")
print(script_template)

---

## Checklist Sebelum Pengumpulan

Pastikan Anda telah:

- [ ] Mengisi nama dan NPM di bagian header
- [ ] Menyelesaikan Bagian 1: memuat data dan plot 5 detik pertama
- [ ] Menyelesaikan Bagian 2: preprocessing manual (3 langkah) dan evaluasi SNR/RMSE
- [ ] Menyelesaikan Bagian 3: implementasi Pan-Tompkins manual dan evaluasi deteksi
- [ ] Menyelesaikan Bagian 4: analisis RR interval dan fitur HRV
- [ ] Menjawab semua pertanyaan analisis (Pertanyaan 1, 2, dan 3)
- [ ] Menjalankan seluruh notebook dari awal (`Kernel > Restart & Run All`) tanpa error
- [ ] Semua plot tampil dengan benar dan memiliki label sumbu serta judul
- [ ] (Bonus) Membuat `ecg_pipeline.py` yang dapat dijalankan dari terminal

---
*Tugas 3 -- Pengolahan Sinyal Medis -- Universitas Indonesia*